In [ ]:
# 我們會使用到一些內建的資料庫, MAC需要加入以下兩行, 才不會把對方的ssl憑證視為無效
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [ ]:
from keras.datasets import mnist
# 回傳值: ((訓練特徵, 訓練目標), (測試特徵, 測試目標))
(x_train, y_train),(x_test, y_test) = mnist.load_data()


In [ ]:
x_train.shape

In [ ]:
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers import BatchNormalization, UpSampling2D, Conv2D
random_dim = 100
generator = Sequential()
# 先讓100隨機亂數可以變成 7 * 7 * 128
# 為何是 7 * 7呢?
# 因為 7 *7 -> (第一次轉置) 14 * 14 -> (第二次轉置) 28 * 28
# 128則是使用類似VGG的概念, 選擇128開始
generator.add(Dense(7 * 7 * 128, input_dim=random_dim, activation='relu'))
# 轉換成三維
generator.add(Reshape((7, 7, 128)))
# 上採樣, 長寬變兩倍
generator.add(UpSampling2D(size=(2, 2)))
# (4, 4)卷積窗的卷積, 之所以做(4, 4)是為了跟discriminator配合, 我們等discriminator再談
generator.add(Conv2D(128, kernel_size=(4, 4), activation='relu', padding='same'))
# 卷積層間我喜歡使用BN來normalize
generator.add(BatchNormalization())
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(4, 4), activation='relu', padding='same'))
generator.add(BatchNormalization())
# 最後讓filter數目回到1, 因為是灰階圖片, 最後輸出 28 * 28 * 1圖片
# 一樣使用tanh(-1 - 1)作為激活
generator.add(Conv2D(1, kernel_size=(4, 4), activation='tanh', padding='same'))
generator.summary()

In [ ]:
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers import BatchNormalization, UpSampling2D, Conv2D
random_dim = 100
generator = Sequential()
# 先讓100隨機亂數可以變成 7 * 7 * 128
# 為何是 7 * 7呢?
# 因為 7 *7 -> (第一次轉置) 14 * 14 -> (第二次轉置) 28 * 28
# 128則是使用類似VGG的概念, 選擇128開始
generator.add(Dense(7 * 7 * 128, input_dim=random_dim, activation='relu'))
# 轉換成三維
generator.add(Reshape((7, 7, 128)))
# 上採樣, 長寬變兩倍
generator.add(UpSampling2D(size=(2, 2)))
# (4, 4)卷積窗的卷積, 之所以做(4, 4)是為了跟discriminator配合, 我們等discriminator再談
generator.add(Conv2D(128, kernel_size=(4, 4), activation='relu', padding='same'))
# 卷積層間我喜歡使用BN來normalize
generator.add(BatchNormalization())
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(4, 4), activation='relu', padding='same'))
generator.add(BatchNormalization())
# 最後讓filter數目回到1, 因為是灰階圖片, 最後輸出 28 * 28 * 1圖片
# 一樣使用tanh(-1 - 1)作為激活
generator.add(Conv2D(1, kernel_size=(4, 4), activation='tanh', padding='same'))
generator.summary()

In [ ]:
from keras.layers import Input
discriminator.trainable = False
gan_input = Input(shape=(random_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(inputs=gan_input, outputs=gan_output)
gan.compile(loss='binary_crossentropy', optimizer="adam")
gan.summary()

In [ ]:
from keras.layers import Input
discriminator.trainable = False
gan_input = Input(shape=(random_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(inputs=gan_input, outputs=gan_output)
gan.compile(loss='binary_crossentropy', optimizer="adam")
gan.summary()

In [ ]:
batch_size = 200
epoch_count = 10
for epoch in range(0, epoch_count):
    for batch_count in range(0, 300):
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train_shaped[idx]

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        # 步驟0:讓創作家製造出fake image
        noise = np.random.normal(0, 1, (batch_size, random_dim))
        gen_imgs = generator.predict(noise)

        discriminator.trainable = True
        # 步驟1:讓鑑賞家鑑賞對的image
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        # 步驟2:讓鑑賞家鑑賞錯的image
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = (d_loss_real + d_loss_fake) / 2

        discriminator.trainable = False
        noise = np.random.normal(0, 1, (batch_size, random_dim))
        # 步驟3:訓練創作家的創作能力
        g_loss = gan.train_on_batch(noise, valid)
    if (epoch + 1) % 10 == 0:
        dash = "-" * 15
        print(dash, "epoch", epoch + 1, dash)
        print("Discriminator loss:", d_loss)
        print("Generator loss:", g_loss)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
examples = 100
noise = np.random.normal(0, 1, (examples, random_dim))
gen_imgs = generator.predict(noise)

# Rescale images 0 - 1
gen_imgs = 0.5 * gen_imgs + 0.5
gen_imgs = gen_imgs.reshape(examples, 28, 28)
plt.figure(figsize = (14, 14))

w = 10
h = int(examples / w) + 1
for i in range(0, examples):
    plt.subplot(h, w, i + 1)
    plt.axis('off')
    plt.imshow(gen_imgs[i], cmap='gray')